In [2]:
import os
import glob
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from utils import *
import logging
# from simpledbf import Dbf5

MX_DIR_2 = '/Users/tszchun.chow/Library/CloudStorage/OneDrive-SharedLibraries-AucklandTransport/Congestion Charging - General/3. Policy & Strategy/Scheme analysis/Modelling outputs/010_OD_Times'

In [14]:
def mx_to_ijk(df, metrics, model_code, mode, trip_purpose):
    df = pd.melt(df, id_vars = ['origin'], 
                value_vars = df.columns.tolist().remove('origin'), 
                var_name = 'destination', 
                value_name = f'{metrics}_{model_code}_{mode}_{trip_purpose}')
    return df

def year(scen_code):
    if scen_code == 12:
        return "18"
    elif scen_code == 20:
        return "31"
    elif scen_code == 22:
        return "41"
    elif scen_code == 24:
        return "51"
    else:
        return str(YEAR)

In [3]:
# SCENARIO_MAP = {'2018': 12,
#                 '2026': 16,
#                 '2031': 20,
#                 '2041': 22,
#                 '2051': 24 
#                 }
# REVERSE_SCENARIO_MAP = {v: k for k, v in SCENARIO_MAP.items()}
# REVERSE_SCENARIO_MAP
# FUTURE_MX_DIR = '/Users/tszchun.chow/Library/CloudStorage/OneDrive-SharedLibraries-AucklandTransport/Congestion Charging - General/3. Policy & Strategy/Scheme analysis/Modelling outputs/011_Strategic_Case/Matrices/Matrices'

# PERIOD_MAP = {'AM': 1, 'IP': 2, 'PM': 3, 'SP': 4, 'OP': 5}
# PERIOD_LIST = [x for x in PERIOD_MAP]
# REVERSE_PERIOD_MAP = {v: k for k, v in PERIOD_MAP.items()}

In [15]:
def map_sector(mx_df, crosswalk = MSM_ZONES):
    mx_df = mx_df.merge(
        crosswalk[['MSM2018', 'LBA_Name', 'Sector_3']].rename(
            columns={'MSM2018': 'origin', 
                     'Sector_3': 'origin_sector', 
                     'LBA_Name': 'origin_LBA'}), 
            on='origin'
    ).merge(
        crosswalk[['MSM2018', 'LBA_Name', 'Sector_3']].rename(
            columns={'MSM2018': 'destination', 
                     'Sector_3': 'destination_sector', 
                     'LBA_Name': 'destination_LBA'}),
            on='destination'
    )
    return mx_df


def get_mx(metrics, period_list = PERIOD_LIST, modes_list = MODES_LIST, trip_purpose_list = ['All'] ,delta = False):

    dfs = []
    if metrics in ('Trips', 'Time', 'Distance', 'Toll', 'Fare', 'GC'):
        counter = 0

        for period in PERIOD_MAP:
            period_code = PERIOD_MAP[period]
            for scenario in SCENARIO_MAP:
                yr = year(SCENARIO_MAP[scenario])
                if scenario in ['2031', '2041', '2051']:
                    scen_code = 16
                    search_location = MX_DIR + f'/{scenario}/*.csv'
                else:
                    scen_code = SCENARIO_MAP[scenario]
                    search_location = MX_DIR + '/*/CSV/*.csv'
                for mode in modes_list:
                    if not ((metrics == 'Toll') & (mode == 'PT')) or ((metrics == 'Fare') & (mode == 'Car')):
                        model_code = yr + str(period_code) + str(SCENARIO_MAP[scenario])
                        if metrics in ['Trips', "GC"]:
                            # if metrics == 'Trips':
                                # trip_purpose_list = ['HBW', 'HBE', 'HBSh', 'EB', 'All']
                            if metrics =="GC":
                                trip_purpose_list = list(set(trip_purpose_list).intersection(['HBW', 'EB', 'Other']))
                                if len(trip_purpose_list) == 0 :
                                    print("None of the trip purposes provided in the trip_purpose_list input has a corresponding GC matrix. Outputting HBW, EB, and Other GC matrices altogether.")
                                    trip_purpose_list = ['HBW', 'EB', 'Other']
                            for trip_purpose in trip_purpose_list:
                                if not ((metrics == 'GC') & (trip_purpose == 'All')):
                                    mx_code = scen_code*100 + MX_MAP[f"{period}_{mode}_{trip_purpose}_{metrics}"]
                                    
                                    for file in glob.glob(search_location):
                                        if str(mx_code) in file:
                                            temp_df = pd.read_csv(file, skiprows=10).rename(columns={'p/q/[val]': 'origin'})
                                            temp_df = mx_to_ijk(temp_df, metrics, model_code, mode, trip_purpose)
                                            temp_df['origin'] = temp_df['origin'].astype(int)
                                            temp_df['destination'] = temp_df['destination'].astype(int)
                                            # temp_df['mode'] = mode
                                            # temp_df['scenario'] = model_code

                                            # dfs.append(temp_df)

                                            if counter == 0:
                                                mx_df = temp_df.copy()
                                            else:
                                                mx_df = mx_df.merge(temp_df, on=['origin', 'destination'])
                                
                                            counter += 1

                        else:
                            mx_code = scen_code*100 + MX_MAP[f"{period}_{mode}_All_{metrics}"]

                            for file in glob.glob(search_location):
                                if str(mx_code) in file:
                                    temp_df = pd.read_csv(file, skiprows=10).rename(columns={'p/q/[val]': 'origin'})
                                    temp_df = mx_to_ijk(temp_df, metrics, model_code, mode, "All")
                                    temp_df['origin'] = temp_df['origin'].astype(int)
                                    temp_df['destination'] = temp_df['destination'].astype(int)
                                    # temp_df['mode'] = mode
                                    # temp_df['scenario'] = model_code

                                    # dfs.append(temp_df)

                                    if counter == 0:
                                        mx_df = temp_df.copy()
                                    else:
                                        mx_df = mx_df.merge(temp_df, on=['origin', 'destination'])
                        
                                    counter += 1

        # mx_df = pd.concat(dfs)
        # mx_df = map_sector(mx_df)
        return mx_df
                     
    else:
        return 'Invalid Input'  # or raise an error

def get_mx_by_tp(metrics, modes_list = MODES_LIST, trip_purpose_list = ['All'] ,congested = False):

    dfs = []
    if metrics in ('Trips', 'Time'):
        counter = 0

        for period in PERIOD_MAP:
            period_code = PERIOD_MAP[period]
            for scenario in SCENARIO_MAP:
                scen_code = SCENARIO_MAP[scenario]
                for mode in modes_list:
                    model_code =  year(scen_code) + str(period_code) + str(scen_code)
                    for trip_purpose in trip_purpose_list:
                        if congested:
                            cong_code = "Congested"
                        else:
                            cong_code = ""
                        mx_code = MX_MAP_2[f"{mode}_{trip_purpose}_{metrics}_{cong_code}"]
                
                        for file in glob.glob(MX_DIR_2 + '/*/*/*/*.csv'):
                            if (str(model_code) in file) and (f'mf{str(mx_code)}' in file):
                                temp_df = pd.read_csv(file, skiprows=9)
                                temp_df = temp_df.rename(columns={temp_df.columns[0]: 'origin'})
                                temp_df = mx_to_ijk(temp_df, metrics, model_code, mode, trip_purpose)
                                temp_df['origin'] = temp_df['origin'].astype(int)
                                temp_df['destination'] = temp_df['destination'].astype(int)
                                # temp_df['mode'] = mode
                                # temp_df['scenario'] = model_code

                                # dfs.append(temp_df)

                                if counter == 0:
                                    mx_df = temp_df.copy()
                                else:
                                    mx_df = mx_df.merge(temp_df, on=['origin', 'destination'])

                                if metrics == 'Time':
                                    column_name = f'{metrics}_{model_code}_{mode}_{trip_purpose}'
                                    # mx_df.loc[mx_df['origin']==mx_df['destination'],column_name] = mx_df.groupby('origin')[column_name].transform(lambda x: x[x>0].min()/2)
                                
                    
                                counter += 1

        # mx_df = pd.concat(dfs)
        mx_df = map_sector(mx_df)
        return mx_df
                     
    else:
        return 'Invalid Input'  # or raise an error
    

In [16]:
def get_travel_time_mx(trip_mx, journeytime_mx, trip_purpose,  modes_list = MODES_LIST, crosswalk = MSM_ZONES):
    agg_dict = {}

    counter = 0
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            if trip_purpose == 'HCV':
                modes_list = ['HCV']
            for mode in modes_list:
                model_code = year(scen_code) + str(period_code) + str(scen_code)
                if trip_purpose=='HCV':
                    trip_code = f'Trips_{model_code}_{mode}_All'
                else:
                    trip_code = f'Trips_{model_code}_{mode}_{trip_purpose}'
                time_code = f'Time_{model_code}_{mode}_{trip_purpose}'

                if (trip_code in trip_mx.columns) and (time_code in journeytime_mx.columns):

                    filtered_trips = trip_mx[['origin', 'destination', trip_code]]
                    filtered_times = journeytime_mx[['origin', 'destination', time_code]]
                    temp_df = filtered_trips.merge(filtered_times, on = ['origin', 'destination'])
                    temp_df[f'PHT_{model_code}_{mode}_{trip_purpose}'] = temp_df[time_code] * temp_df[trip_code]
                    # temp_df = temp_df.drop(columns=[trip_code, time_code])

                    if counter == 0:
                        pht_df = temp_df.copy()
                    else:
                        pht_df = pht_df.merge(temp_df, on=['origin', 'destination'])

                    trip_code = f'Trips_{model_code}_{mode}_{trip_purpose}'
                    pht_code = f'PHT_{model_code}_{mode}_{trip_purpose}'
                    agg_dict[trip_code] = 'sum'
                    agg_dict[pht_code] = 'sum'

                    counter += 1

    pht_df = map_sector(pht_df, crosswalk=crosswalk)
    return pht_df

def get_agg_dict(df, trip_purpose, modes_list = MODES_LIST, pht = True):
    agg_dict = {}
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = year(scen_code) + str(period_code) + str(scen_code)

            if trip_purpose != 'HCV':

                for mode in modes_list:
                    trip_code = f'Trips_{model_code}_{mode}_{trip_purpose}'
                    if trip_code in df.columns:
                        pht_code = f'PHT_{model_code}_{mode}_{trip_purpose}'
                        agg_dict[trip_code] = 'sum'
                        if pht:
                            agg_dict[pht_code] = 'sum'
            else:
                mode = 'HCV'
                trip_code = f'Trips_{model_code}_{mode}_All'
                if trip_code in df.columns:
                    pht_code = f'PHT_{model_code}_{mode}_{trip_purpose}'
                    agg_dict[trip_code] = 'sum'
                    if pht:
                        agg_dict[pht_code] = 'sum'
    return agg_dict


In [17]:
all_trip_mx = get_mx('Trips')
# all_time_mx = get_mx('Time')
all_distance_mx = get_mx('Distance')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_93465/3475296168.py", line 1, in <module>
    all_trip_mx = get_mx('Trips')
                  ^^^^^^^^^^^^^^^
  File "/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_93465/1903583274.py", line 63, in get_mx
    mx_df = mx_df.merge(temp_df, on=['origin', 'destination'])
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pandas/core/frame.py", line 10832, in merge
    return merge(
           ^^^^^^
  File "/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/panda

In [33]:
all_veh_trip_mx = get_mx('Trips', modes_list=['Vehicle'])

In [37]:
all_veh_time_mx = all_time_mx.copy()
all_veh_time_mx.columns = all_veh_time_mx.columns.str.replace("_Car", "_Vehicle")
all_veh_time_mx

,origin,destination,Time_18112_Vehicle_All,Time_18112_PT_All,Time_26116_Vehicle_All,Time_26116_PT_All,Time_26176_Vehicle_All,Time_26176_PT_All,Time_26150_Vehicle_All,Time_26150_PT_All,...,Time_26370_Vehicle_All,Time_26370_PT_All,Time_26372_Vehicle_All,Time_26372_PT_All,Time_26378_Vehicle_All,Time_26378_PT_All,Time_26380_Vehicle_All,Time_26380_PT_All,Time_26382_Vehicle_All,Time_26382_PT_All
0,1,1,11.136314,0.0,11.688956,0.0,11.688981,0.0,11.688304,0.0,...,11.623383,0.0,11.616477,0.0,11.616598,0.0,11.620839,0.0,11.621500,0.0
1,2,1,24.957365,0.0,25.813732,0.0,25.813648,0.0,25.811533,0.0,...,25.903666,0.0,25.911619,0.0,25.905495,0.0,25.908604,0.0,25.909227,0.0
2,3,1,26.328798,0.0,27.190876,0.0,27.190758,0.0,27.188990,0.0,...,26.922766,0.0,26.919027,0.0,26.922762,0.0,26.923525,0.0,26.923630,0.0
3,4,1,27.384047,0.0,28.266777,0.0,28.266678,0.0,28.265053,0.0,...,28.139458,0.0,28.136147,0.0,28.139315,0.0,28.140020,0.0,28.140131,0.0
4,5,1,49.359303,0.0,50.134605,0.0,50.136150,0.0,50.131889,0.0,...,51.760635,0.0,51.777580,0.0,51.770428,0.0,51.761936,0.0,51.762665,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489995,99992,99996,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
489996,99993,99996,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
489997,99994,99996,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
489998,99995,99996,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [40]:
all_veh_pht_mx = get_travel_time_mx(all_veh_trip_mx, all_veh_time_mx, trip_purpose='All', modes_list = ['Vehicle'], crosswalk=SECTOR_MSMZONE_MAP)

In [24]:
# domin_all_trip_mx = get_mx('Trips', modes_list=['Vehicle'])
# domin_all_time_mx = get_mx('Time', modes_list=['Car', 'PT'])

# domin_all_pht_mx = get_travel_time_mx(domin_all_trip_mx, domin_all_time_mx, trip_purpose='All', modes_list = ['Vehicle'])
# all_pht_sector_mx = domin_all_pht_mx.groupby(['origin_LBA', 'destination_LBA']
#                                        ).agg(get_agg_dict(domin_all_pht_mx, 'All', modes_list=['Vehicle']))
all_pht_sector_ijk = pd.melt(all_pht_sector_mx.reset_index(),
                             id_vars = ['origin_LBA', 'destination_LBA'],
                             value_vars = all_pht_sector_mx.columns)

all_pht_sector_ijk['Metric'] = all_pht_sector_ijk['variable'].str.split("_").str[0]
all_pht_sector_ijk['Model'] = all_pht_sector_ijk['variable'].str.split("_").str[1]
all_pht_sector_ijk['Period_Code'] = all_pht_sector_ijk['Model'].str[2].astype(int)
all_pht_sector_ijk['Period'] = all_pht_sector_ijk['Period_Code'].map(REVERSE_PERIOD_MAP)
all_pht_sector_ijk['Scenario_Code'] = all_pht_sector_ijk['Model'].str[3:]
all_pht_sector_ijk['Mode'] = all_pht_sector_ijk['variable'].str.split("_").str[2]
all_pht_sector_ijk['Trip_Purpose'] = all_pht_sector_ijk['variable'].str.split("_").str[3]
all_pht_sector_ijk.to_csv(os.path.join(THROUGHPUT_METRICS_DIR, 'FY_pht_byLBA.csv'), index=None)

18112
26116
31120
41122
51124
18212
26216
31220
41222
51224
18312
26316
31320
41322
51324


In [19]:
hbw_trip_mx = get_mx('Trips', trip_purpose_list = ['HBW'])
hbw_time_mx = get_mx('Time', trip_purpose_list=['HBW'])

In [ ]:
all_pht_sector_mx = all_pht_mx.groupby(['origin_sector', 'destination_sector']
                                       ).agg(get_agg_dict(all_pht_mx, 'All'))
all_pht_sector_ijk = pd.melt(all_pht_sector_mx.reset_index(),
                             id_vars = ['origin_sector', 'destination_sector'],
                             value_vars = all_pht_sector_mx.columns)
all_pht_sector_ijk = export_pht_ijk(all_pht_sector_ijk)

In [31]:
all_pht_mx = get_travel_time_mx(all_trip_mx, all_time_mx, trip_purpose='All', crosswalk=SECTOR_MSMZONE_MAP)

In [41]:
def export_pht_ijk(all_pht_sector_ijk, filename = 'pht_bysector.csv'):
    all_pht_sector_ijk['Metric'] = all_pht_sector_ijk['variable'].str.split("_").str[0]
    all_pht_sector_ijk['Model'] = all_pht_sector_ijk['variable'].str.split("_").str[1]
    all_pht_sector_ijk['Period_Code'] = all_pht_sector_ijk['Model'].str[2].astype(int)
    all_pht_sector_ijk['Period'] = all_pht_sector_ijk['Period_Code'].map(REVERSE_PERIOD_MAP)
    all_pht_sector_ijk['Scenario_Code'] = all_pht_sector_ijk['Model'].str[3:]
    all_pht_sector_ijk['Mode'] = all_pht_sector_ijk['variable'].str.split("_").str[2]
    all_pht_sector_ijk['Trip_Purpose'] = all_pht_sector_ijk['variable'].str.split("_").str[3]
    all_pht_sector_ijk.to_csv(os.path.join(THROUGHPUT_METRICS_DIR, filename), index=None)
    return all_pht_sector_ijk

all_pht_lba_mx = all_pht_mx.groupby(['origin_LBA', 'destination_LBA']
                                       ).agg(get_agg_dict(all_pht_mx, 'All'))
all_pht_lba_ijk = pd.melt(all_pht_lba_mx.reset_index(),
                             id_vars = ['origin_LBA', 'destination_LBA'],
                             value_vars = all_pht_lba_mx.columns)

all_pht_lba_ijk = export_pht_ijk(all_pht_lba_ijk, filename='pht_byLBA.csv')

all_veh_pht_lba_mx = all_veh_pht_mx.groupby(['origin_LBA', 'destination_LBA']
                                       ).agg(get_agg_dict(all_veh_pht_mx, 'All', modes_list=['Vehicle']))
all_veh_pht_lba_ijk = pd.melt(all_veh_pht_lba_mx.reset_index(),
                             id_vars = ['origin_LBA', 'destination_LBA'],
                             value_vars = all_veh_pht_lba_mx.columns)

all_veh_pht_lba_ijk = export_pht_ijk(all_veh_pht_lba_ijk, filename='veh_pht_byLBA.csv')

In [6]:
hbw_time_mx = get_mx_by_tp('Time', trip_purpose_list=['HBW'])
hbsh_time_mx = get_mx_by_tp('Time', trip_purpose_list=['HBSh'])
eb_time_mx = get_mx_by_tp('Time', trip_purpose_list=['EB'])
freight_time_mx = get_mx_by_tp('Time', modes_list= ['HCV'], trip_purpose_list=['HCV'])

KeyboardInterrupt: 

In [7]:
hbw_trip_mx = get_mx('Trips', trip_purpose_list = ['HBW'])
hbsh_trip_mx = get_mx('Trips', trip_purpose_list = ['HBSh'])
eb_trip_mx = get_mx('Trips', trip_purpose_list = ['EB'])
# freight_trip_mx = get_mx('Trips', modes_list=['HCV'])

In [9]:
SECTOR_MSMZONE_MAP = pd.read_csv(INPUT_DIR + '/msm_zones/msmzone_lba_sector_crosswalk.csv')[['MSM2018', 'LBA_Name', 'Sector_3']]
origin_zone_map = SECTOR_MSMZONE_MAP.rename(columns={'MSM2018': 'origin', 'LBA_Name': 'origin_LBA', 'Sector_3': 'origin_sector'})
destination_zone_map = SECTOR_MSMZONE_MAP.rename(columns={'MSM2018': 'destination', 'LBA_Name': 'destination_LBA', 'Sector_3': 'destination_sector'})

origin_sector_map = origin_zone_map[['origin_LBA', 'origin_sector']].drop_duplicates()
destination_sector_map = destination_zone_map[['destination_LBA', 'destination_sector']].drop_duplicates()

In [9]:
hbw_trip_mapped_mx = hbw_trip_mx.merge(origin_zone_map, on='origin').merge(destination_zone_map, on='destination')
hbsh_trip_mapped_mx = hbsh_trip_mx.merge(origin_zone_map, on='origin').merge(destination_zone_map, on='destination')
eb_trip_mapped_mx = eb_trip_mx.merge(origin_zone_map, on='origin').merge(destination_zone_map, on='destination')

hbw_dailytrip_od_sector_df = hbw_trip_mapped_mx.groupby(['origin_sector', 'destination_sector']).agg(get_agg_dict(hbw_trip_mapped_mx, 'HBW', pht=False)).reset_index()
hbsh_dailytrip_od_sector_df = hbsh_trip_mapped_mx.groupby(['origin_sector', 'destination_sector']).agg(get_agg_dict(hbsh_trip_mapped_mx, 'HBSh', pht=False)).reset_index()
eb_dailytrip_od_sector_df = eb_trip_mapped_mx.groupby(['origin_sector', 'destination_sector']).agg(get_agg_dict(eb_trip_mapped_mx, 'EB', pht=False)).reset_index()

In [25]:
dailytrip_od_df = hbw_dailytrip_od_sector_df.merge(
    hbsh_dailytrip_od_sector_df, on=['origin_sector', 'destination_sector']
    ).merge(eb_dailytrip_od_sector_df, on=['origin_sector', 'destination_sector']
    )
value_vars = dailytrip_od_df.set_index(['origin_sector', 'destination_sector']).columns
dailytrip_od_ijk = pd.melt(dailytrip_od_df, 
        id_vars = ['origin_sector', 'destination_sector'],
        value_vars = value_vars)
dailytrip_od_ijk['metric'] = dailytrip_od_ijk['variable'].str.split("_").str[0]
dailytrip_od_ijk['model_code'] = dailytrip_od_ijk['variable'].str.split("_").str[1]
dailytrip_od_ijk['mode'] = dailytrip_od_ijk['variable'].str.split("_").str[2]
dailytrip_od_ijk['trip_purpose'] = dailytrip_od_ijk['variable'].str.split("_").str[3]

dailytrip_od_ijk['period_code'] = dailytrip_od_ijk['model_code'].str[2]
dailytrip_od_ijk['scenario_code'] = dailytrip_od_ijk['model_code'].str[-2:]
dailytrip_od_ijk['period'] = dailytrip_od_ijk['period_code'].astype(int).map(REVERSE_PERIOD_MAP)
dailytrip_od_ijk['scenario'] = dailytrip_od_ijk['scenario_code'].astype(int).map(REVERSE_SCENARIO_MAP)
dailytrip_od_ijk = dailytrip_od_ijk.merge(origin_sector_map, on='origin_sector'
                                          ).merge(destination_sector_map, on='destination_sector')
dailytrip_od_ijk.to_csv(f'{THROUGHPUT_METRICS_DIR}/DailyTrips_OD_ijk.csv', index=None)

In [16]:
dailytrip_od_ijk

,origin_sector,destination_sector,variable,value,metric,model_code,mode,trip_purpose,period_code,scenario_code,period,scenario
0,Albany,Albany,Trips_18112_Car_HBW,2741.804128,Trips,18112,Car,HBW,1,12,AM,2018
1,Albany,CBD,Trips_18112_Car_HBW,305.418985,Trips,18112,Car,HBW,1,12,AM,2018
2,Albany,City Fringe East,Trips_18112_Car_HBW,108.116879,Trips,18112,Car,HBW,1,12,AM,2018
3,Albany,City Fringe West,Trips_18112_Car_HBW,57.716099,Trips,18112,Car,HBW,1,12,AM,2018
4,Albany,Devonport - Takapuna,Trips_18112_Car_HBW,468.079596,Trips,18112,Car,HBW,1,12,AM,2018
...,...,...,...,...,...,...,...,...,...,...,...,...
431995,Whau,Puketapapa,Trips_26582_PT_EB,0.851403,Trips,26582,PT,EB,5,82,OP,Option 3G
431996,Whau,Remuera / Ellerslie,Trips_26582_PT_EB,0.405955,Trips,26582,PT,EB,5,82,OP,Option 3G
431997,Whau,Rodney,Trips_26582_PT_EB,0.188683,Trips,26582,PT,EB,5,82,OP,Option 3G
431998,Whau,Waitakere Ranges,Trips_26582_PT_EB,0.857548,Trips,26582,PT,EB,5,82,OP,Option 3G


In [ ]:
hbw_totalhours_df = get_travel_time_mx(hbw_trip_mx, hbw_time_mx, 'HBW')
hbsh_totalhours_df = get_travel_time_mx(hbsh_trip_mx, hbsh_time_mx, 'HBSh')
eb_totalhours_df = get_travel_time_mx(eb_trip_mx, eb_time_mx, 'EB')
freight_totalhours_df = get_travel_time_mx(freight_trip_mx, freight_time_mx, 'HCV')

UnboundLocalError: cannot access local variable 'pht_df' where it is not associated with a value

In [10]:
def get_agg_dict(trip_purpose):
    agg_dict = {}
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = year(scen_code) + str(period_code) + str(scen_code)
            if scen_code > 16:
                if trip_purpose != 'HCV':
                    modes_list = MODES_LIST
                    for mode in modes_list:
                        trip_code = f'Trips_{model_code}_{mode}_{trip_purpose}'
                        pht_code = f'PHT_{model_code}_{mode}_{trip_purpose}'
                        agg_dict[trip_code] = 'sum'
                        agg_dict[pht_code] = 'sum'
                else:
                    mode = 'HCV'
                    trip_code = f'Trips_{model_code}_{mode}_All'
                    pht_code = f'PHT_{model_code}_{mode}_{trip_purpose}'
                    agg_dict[trip_code] = 'sum'
                    agg_dict[pht_code] = 'sum'
    return agg_dict


In [102]:
hbw_origin_sector_df = hbw_totalhours_df.groupby(['origin_sector']).agg(get_agg_dict('HBW')).reset_index()
hbsh_origin_sector_df = hbsh_totalhours_df.groupby(['origin_sector']).agg(get_agg_dict('HBSh')).reset_index()
eb_origin_sector_df = eb_totalhours_df.groupby(['origin_sector']).agg(get_agg_dict('EB')).reset_index()
freight_origin_sector_df = freight_totalhours_df.groupby(['origin_sector']).agg(get_agg_dict('HCV')).reset_index()
freight_origin_sector_df.columns = freight_origin_sector_df.columns.str.replace('_HCV_All', '_HCV_HCV') # reset_index()

In [125]:
hbw_destination_sector_df = hbw_totalhours_df.groupby(['destination_sector']).agg(get_agg_dict('HBW')).reset_index()
hbsh_destination_sector_df = hbsh_totalhours_df.groupby(['destination_sector']).agg(get_agg_dict('HBSh')).reset_index()
eb_destination_sector_df = eb_totalhours_df.groupby(['destination_sector']).agg(get_agg_dict('EB')).reset_index()
freight_destination_sector_df = freight_totalhours_df.groupby(['destination_sector']).agg(get_agg_dict('HCV')).reset_index()
freight_destination_sector_df.columns = freight_destination_sector_df.columns.str.replace('_HCV_All', '_HCV_HCV') # reset_index()


In [ ]:
hbw_od_sector_df = hbw_totalhours_df.groupby(['origin_sector', 'destination_sector']).agg(get_agg_dict('HBW')).reset_index()
hbsh_od_sector_df = hbsh_totalhours_df.groupby(['origin_sector', 'destination_sector']).agg(get_agg_dict('HBSh')).reset_index()
eb_od_sector_df = eb_totalhours_df.groupby(['origin_sector', 'destination_sector']).agg(get_agg_dict('EB')).reset_index()
freight_od_sector_df = freight_totalhours_df.groupby(['origin_sector', 'destination_sector']).agg(get_agg_dict('HCV')).reset_index()
freight_od_sector_df.columns = freight_od_sector_df.columns.str.replace('_HCV_All', '_HCV_HCV') # reset_index()

In [113]:
sectors_gdf = MSM_ZONES.dissolve('Sector_3')[['LBA_Name', 'geometry']].reset_index().to_crs('epsg:4326').rename(columns={'Sector_3':'Sector'})

In [127]:
local_economy_od_metrics_df = hbw_od_sector_df.merge(
    hbsh_od_sector_df, on=['origin_sector', 'destination_sector']
    ).merge(eb_od_sector_df, on=['origin_sector', 'destination_sector']
    ).merge(freight_od_sector_df, on=['origin_sector', 'destination_sector'])

In [130]:
local_economy_od_metrics_df.to_parquet(f'{THROUGHPUT_METRICS_DIR}/JT_OD_TP.parquet')
local_economy_od_metrics_df.to_csv(f'{THROUGHPUT_METRICS_DIR}/JT_OD_TP.csv', index=None)

In [94]:
lba_sector_lookup = SECTOR_MSMZONE_MAP[['LBA_Name', 'Sector_3']].drop_duplicates()
lba_sector_lookup_origin = lba_sector_lookup.rename(columns={'LBA_Name':' origin_LBA', 'Sector_3': 'origin_sector'})
lba_sector_lookup_destination = lba_sector_lookup.rename(columns={'LBA_Name':' destination_LBA', 'Sector_3': 'destination_sector'})

In [96]:
local_economy_od_metrics_df = pd.read_parquet(f'{THROUGHPUT_METRICS_DIR}/JT_OD_TP.parquet')
value_vars = local_economy_od_metrics_df.set_index(['origin_sector', 'destination_sector']).columns
local_economy_metrics_ijk = pd.melt(local_economy_od_metrics_df, 
        id_vars = ['origin_sector', 'destination_sector'],
        value_vars = value_vars)
local_economy_metrics_ijk['metric'] = local_economy_metrics_ijk['variable'].str.split("_").str[0]
local_economy_metrics_ijk['model_code'] = local_economy_metrics_ijk['variable'].str.split("_").str[1]
local_economy_metrics_ijk['mode'] = local_economy_metrics_ijk['variable'].str.split("_").str[2]
local_economy_metrics_ijk['trip_purpose'] = local_economy_metrics_ijk['variable'].str.split("_").str[3]

local_economy_metrics_ijk['period_code'] = local_economy_metrics_ijk['model_code'].str[2]
local_economy_metrics_ijk['scenario_code'] = local_economy_metrics_ijk['model_code'].str[-2:]
local_economy_metrics_ijk['period'] = local_economy_metrics_ijk['period_code'].astype(int).map(REVERSE_PERIOD_MAP)
local_economy_metrics_ijk['scenario'] = local_economy_metrics_ijk['scenario_code'].astype(int).map(REVERSE_SCENARIO_MAP)
local_economy_metrics_ijk = local_economy_metrics_ijk.merge(lba_sector_lookup_origin, on='origin_sector').merge(lba_sector_lookup_destination, on='destination_sector')
local_economy_metrics_ijk.to_csv(f'{THROUGHPUT_METRICS_DIR}/JT_OD_TP_ijk.csv', index=None)


In [115]:
local_economy_metric_gdf = sectors_gdf.merge(hbw_origin_sector_df.merge(
    hbsh_origin_sector_df, on=['origin_sector']
    ).merge(eb_origin_sector_df, on=['origin_sector']
    ).merge(freight_origin_sector_df, on=['origin_sector']), left_on='Sector', right_on='origin_sector')

In [139]:
value_vars = local_economy_metrics_gdf.set_index(['LBA_Name', 'origin_sector', 'Sector', 'geometry', 'wkt']).columns

In [150]:
REVERSE_PERIOD_MAP = {v: k for k, v in PERIOD_MAP.items()}
REVERSE_SCENARIO_MAP = {v: k for k, v in SCENARIO_MAP.items()}

{12: '2018',
 16: 'Do Minimum',
 76: 'Do Minimum 3C',
 50: 'Option 1A',
 52: 'Option 1B',
 54: 'Option 1C',
 56: 'Option 2A',
 58: 'Option 2B',
 64: 'Option 3A',
 66: 'Option 3B',
 74: 'Option 3C',
 70: 'Option 3D',
 72: 'Option 3E'}

In [153]:
local_economy_metrics_gdf.set_index(['LBA_Name', 'origin_sector'])

local_economy_metrics_ijk = pd.melt(local_economy_metrics_gdf.drop(columns=['geometry', 'wkt', 'Sector']), 
        id_vars = ['LBA_Name', 'origin_sector'],
        value_vars = value_vars)

local_economy_metrics_ijk['metric'] = local_economy_metrics_ijk['variable'].str.split("_").str[0]
local_economy_metrics_ijk['model_code'] = local_economy_metrics_ijk['variable'].str.split("_").str[1]
local_economy_metrics_ijk['mode'] = local_economy_metrics_ijk['variable'].str.split("_").str[2]
local_economy_metrics_ijk['trip_purpose'] = local_economy_metrics_ijk['variable'].str.split("_").str[3]

local_economy_metrics_ijk['period_code'] = local_economy_metrics_ijk['model_code'].str[2]
local_economy_metrics_ijk['scenario_code'] = local_economy_metrics_ijk['model_code'].str[-2:]
local_economy_metrics_ijk['period'] = local_economy_metrics_ijk['period_code'].astype(int).map(REVERSE_PERIOD_MAP)
local_economy_metrics_ijk['scenario'] = local_economy_metrics_ijk['scenario_code'].astype(int).map(REVERSE_SCENARIO_MAP)
local_economy_metrics_ijk.to_csv(f'{THROUGHPUT_METRICS_DIR}/JT_Origin_TP.csv', index=None)

In [124]:
local_economy_metrics_destination_gdf = sectors_gdf.merge(hbw_destination_sector_df.merge(
    hbsh_destination_sector_df, on='destination_sector'
    ).merge(eb_destination_sector_df, on='destination_sector'
    ).merge(freight_destination_sector_df, on='destination_sector'), left_on='Sector', right_on='destination_sector')

KeyError: 'destination_sector'

In [116]:
def export_geoparquet(geo_package, output_path, filename, id = 'ID'):
    geo_package.to_crs("EPSG:4326", inplace=True)
    geo_package['wkt'] = geo_package['geometry'].to_wkt()
    
    geo_dataframe = gpd.GeoDataFrame()
    geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])    
    geo_dataframe = gpd.GeoDataFrame(geo_dataframe, geometry="geometry")
    geo_dataframe[id] = geo_package[id]

    geo_parquet = geo_dataframe.merge(geo_package[[i for i in geo_package.columns if i not in ["geometry", "wkt"]]],
                                    left_on=id, right_on = id)

    geo_parquet = geo_parquet.dropna(subset=['geometry'])
    if output_path is not None and filename is not None:
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        geo_parquet.fillna(0, inplace=True)
        geo_parquet.to_parquet(f'{output_path}/{filename}', compression = 'zstd')

export_geoparquet(local_economy_metrics_gdf, output_path=THROUGHPUT_METRICS_DIR, filename='JourneyTime_Origin_byTripPurpose.parquet', id = 'origin_sector')

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_45200/981818591.py:6: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])


In [117]:
local_economy_metrics_gdf

,Sector,LBA_Name,geometry,origin_sector,Trips_26176_Car_HBW,PHT_26176_Car_HBW,Trips_26176_PT_HBW,PHT_26176_PT_HBW,Trips_26150_Car_HBW,PHT_26150_Car_HBW,...,PHT_26364_HCV_HCV,Trips_26366_HCV_HCV,PHT_26366_HCV_HCV,Trips_26374_HCV_HCV,PHT_26374_HCV_HCV,Trips_26370_HCV_HCV,PHT_26370_HCV_HCV,Trips_26372_HCV_HCV,PHT_26372_HCV_HCV,wkt
0,Albany,Upper Harbour,"MULTIPOLYGON (((174.70282 -36.77142, 174.70245...",Albany,6044.188542,98043.825158,1589.830023,35728.784005,5977.008106,92984.163968,...,23938.107586,1309.781084,26260.414060,1309.781084,26517.389315,1309.781084,30047.993599,1309.781084,32393.995365,"MULTIPOLYGON (((174.70282 -36.771423, 174.7024..."
1,CBD,Waitemata,"POLYGON ((174.7659 -36.85894, 174.76544 -36.85...",CBD,2346.580199,36582.163853,1347.346233,23351.942411,2258.426441,35252.574555,...,40358.050237,1588.732284,45506.344975,1588.732284,40064.452601,1588.732284,46916.387303,1588.732284,48670.319414,"POLYGON ((174.765896 -36.858944, 174.765441 -3..."
2,City Fringe East,Waitemata,"POLYGON ((174.77827 -36.87045, 174.77784 -36.8...",City Fringe East,2141.001865,30260.472718,1377.020076,17807.063664,2081.191111,30647.709870,...,10529.449792,369.541705,11256.833531,369.541705,10780.082766,369.541705,11401.721341,369.541705,11053.998354,"POLYGON ((174.778268 -36.870451, 174.777842 -3..."
3,City Fringe West,Waitemata,"POLYGON ((174.74147 -36.86881, 174.74011 -36.8...",City Fringe West,3479.042874,51450.756294,2389.744104,33650.425119,3343.693607,50658.898486,...,14602.929105,564.843161,16188.298116,564.843161,16580.674308,564.843161,17113.971708,564.843161,17146.655376,"POLYGON ((174.741468 -36.86881, 174.740105 -36..."
4,Devonport - Takapuna,Devonport - Takapuna,"POLYGON ((174.76556 -36.79729, 174.76427 -36.7...",Devonport - Takapuna,6101.349057,123149.264211,2780.323755,49784.363699,6013.938059,115702.187542,...,15601.636631,504.109930,16026.728499,504.109930,16185.796770,504.109930,19322.700256,504.109930,19812.691483,"POLYGON ((174.765557 -36.797287, 174.764273 -3..."
5,East Tamaki / Flat Bush,Howick,"POLYGON ((174.89604 -36.98668, 174.89592 -36.9...",East Tamaki / Flat Bush,7664.471115,146549.236569,983.704621,40378.946353,7668.364797,145811.619362,...,46540.013350,2430.383869,45703.057336,2430.383869,45688.822758,2430.383869,46148.575854,2430.383869,46423.969872,"POLYGON ((174.896041 -36.986677, 174.895918 -3..."
6,Franklin,Franklin,"MULTIPOLYGON (((174.77827 -37.24119, 174.77749...",Franklin,13637.639161,405198.183285,3039.618306,145409.249654,13629.073689,406043.462981,...,35085.607297,1731.332578,34782.928822,1731.332578,34777.994559,1731.332578,34862.535924,1731.332578,34998.083863,"MULTIPOLYGON (((174.778273 -37.241193, 174.777..."
7,Henderson,Henderson - Massey,"POLYGON ((174.62268 -36.90017, 174.62244 -36.9...",Henderson,10361.919288,267293.331001,3162.305937,101318.643576,10171.480837,250212.301090,...,18121.554409,832.559765,19419.920826,832.559765,19540.762271,832.559765,19588.254613,832.559765,20397.331377,"POLYGON ((174.622676 -36.900169, 174.622441 -3..."
8,Hibiscus Coast,Hibiscus and Bays,"MULTIPOLYGON (((174.66293 -36.65034, 174.66264...",Hibiscus Coast,7831.073780,208037.525606,1274.730019,51067.459448,7811.176392,201999.886397,...,15999.409759,753.008431,16741.990339,753.008431,16825.426139,753.008431,17749.314818,753.008431,18293.459572,"MULTIPOLYGON (((174.662926 -36.650343, 174.662..."
9,Hobsonville,Upper Harbour,"POLYGON ((174.64062 -36.81098, 174.64066 -36.8...",Hobsonville,2766.330869,69397.986448,427.077005,17687.002516,2768.853685,66570.632157,...,12790.496990,674.844122,13001.781883,674.844122,13032.670725,674.844122,13089.061960,674.844122,13168.293369,"POLYGON ((174.640615 -36.810976, 174.640662 -3..."


# Generalised Costs

In [ ]:
hbw_trip_mx = get_mx('Trips', trip_purpose_list = ['HBW'])
hbw_time_mx = get_mx('Time', trip_purpose_list=['HBW'])

In [11]:
hbw_gc_mx = get_mx('GC', trip_purpose_list=['HBW'])

In [13]:
hbw_gc_mx

,origin,destination,GC_26116_Car_HBW,GC_26116_PT_HBW,GC_26176_Car_HBW,GC_26176_PT_HBW,GC_26150_Car_HBW,GC_26150_PT_HBW,GC_26152_Car_HBW,GC_26152_PT_HBW,...,GC_26370_Car_HBW,GC_26370_PT_HBW,GC_26372_Car_HBW,GC_26372_PT_HBW,GC_26378_Car_HBW,GC_26378_PT_HBW,GC_26380_Car_HBW,GC_26380_PT_HBW,GC_26382_Car_HBW,GC_26382_PT_HBW
0,1,1,15.769327,4999.5,15.769352,4999.5,15.768675,4999.5,15.767461,4999.5,...,15.703754,4999.5,15.696848,4999.5,15.696969,4999.5,15.701210,4999.5,15.701872,4999.5
1,2,1,35.816154,9999.0,35.816071,9999.0,35.813950,9999.0,35.823601,9999.0,...,35.906090,9999.0,35.914040,9999.0,35.907913,9999.0,35.911026,9999.0,35.911644,9999.0
2,3,1,37.991051,9999.0,37.990936,9999.0,37.989162,9999.0,37.998497,9999.0,...,37.722946,9999.0,37.719208,9999.0,37.722939,9999.0,37.723705,9999.0,37.723808,9999.0
3,4,1,40.083981,9999.0,40.083881,9999.0,40.082260,9999.0,40.091423,9999.0,...,39.956665,9999.0,39.953354,9999.0,39.956520,9999.0,39.957222,9999.0,39.957340,9999.0
4,5,1,71.641716,9999.0,71.643257,9999.0,71.638992,9999.0,71.624344,9999.0,...,73.267746,9999.0,73.284691,9999.0,73.277527,9999.0,73.269035,9999.0,73.269775,9999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489995,99992,99996,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,...,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0
489996,99993,99996,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,...,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0
489997,99994,99996,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,...,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0
489998,99995,99996,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,...,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0,0.000000,9999.0


In [12]:
LANDUSE_ASSUMPTIONS = pd.read_csv("/Users/tszchun.chow/Documents/GitHub/akl_touch/inputs/land_use_assumptions/Landuse_Detailed_Table_2026_Slot_16.csv", 
                                  skiprows=10).drop(columns='c ')[:-1]
LANDUSE_ASSUMPTIONS_2018 = pd.read_csv("/Users/tszchun.chow/Documents/GitHub/akl_touch/inputs/land_use_assumptions/Landuse_Detailed_Table_2018_Slot_12.csv", 
                                  skiprows=10).drop(columns='c ')[:-1]


employment_cols = [x for x in LANDUSE_ASSUMPTIONS.columns.tolist() if "E" in x]
LANDUSE_ASSUMPTIONS['E'] = LANDUSE_ASSUMPTIONS[employment_cols].sum(axis=1)
employment_assumptions = LANDUSE_ASSUMPTIONS[['Zone', 'E']]

population_cols = [x for x in LANDUSE_ASSUMPTIONS.columns.tolist() if "PT" in x]
LANDUSE_ASSUMPTIONS['Population'] = LANDUSE_ASSUMPTIONS[population_cols].sum(axis=1)
population_assumptions = LANDUSE_ASSUMPTIONS[['Zone', 'Population']]

LANDUSE_ASSUMPTIONS_2018 = pd.read_csv("/Users/tszchun.chow/Documents/GitHub/akl_touch/inputs/land_use_assumptions/Landuse_Detailed_Table_2018_Slot_12.csv", 
                                  skiprows=10).drop(columns='c ')[:-1]

employment_cols_2018 = [x for x in LANDUSE_ASSUMPTIONS_2018.columns.tolist() if "E" in x]
LANDUSE_ASSUMPTIONS_2018['E'] = LANDUSE_ASSUMPTIONS_2018[employment_cols_2018].sum(axis=1)
employment_assumptions_2018 = LANDUSE_ASSUMPTIONS_2018[['Zone', 'E']]

population_cols_2018 = [x for x in LANDUSE_ASSUMPTIONS_2018.columns.tolist() if "PT" in x]
LANDUSE_ASSUMPTIONS_2018['Population'] = LANDUSE_ASSUMPTIONS_2018[population_cols].sum(axis=1)
population_assumptions_2018 = LANDUSE_ASSUMPTIONS_2018[['Zone', 'Population']]

In [14]:
def get_job_accessibility(mx, employment_assumptions, metric = 'Time', trip_purpose = 'All'):
      jt_emp_mx = mx[['origin', 'destination'] + [x for x in mx.columns if trip_purpose in x]
                        ].merge(employment_assumptions.rename(columns={'Zone': 'origin'}), on='origin'
                              ).rename(columns={'E': 'E_origin'}
                                    ).merge(employment_assumptions.rename(columns={'Zone': 'destination'}), on='destination'
                                          ).rename(columns={'E': 'E_destination'})

      agg_access_dict = {} 

      for period in PERIOD_MAP:
            period_code = PERIOD_MAP[period]
            for scenario in SCENARIO_MAP:
                  scen_code = SCENARIO_MAP[scenario]
                  if scen_code != 0:
                        for mode in MODES_LIST:
                              if mode == 'Car':
                                    jt_emp_mx = jt_emp_mx[~((jt_emp_mx['origin']==596)|(jt_emp_mx['destination']==596))]
                              model_code = year(scen_code) + str(period_code) + str(scen_code)
                              time_code = f"{metric}_{model_code}_{mode}_{trip_purpose}"
                              if time_code in jt_emp_mx.columns:
                                    jt_emp_mx.loc[((jt_emp_mx['origin'] == jt_emp_mx['destination'])&(jt_emp_mx[time_code]==0)), time_code] = (jt_emp_mx.groupby(['origin'])[time_code].transform(lambda x: x[x != 0].min()))/2
                                    jt_emp_mx[time_code] = jt_emp_mx[time_code].replace(0, 99999).fillna(99999)
                                    access_code = f"AccToEmp-{metric}_{model_code}_{mode}_{trip_purpose}"

                                    if mode == 'Car':
                                          ceil = 30
                                    else:
                                          ceil = 45

                                    jt_emp_mx.loc[jt_emp_mx[time_code] <= ceil, access_code] = jt_emp_mx['E_destination']
                                    jt_emp_mx.loc[jt_emp_mx[time_code] > ceil, access_code] = 0

                                    agg_access_dict[access_code] = 'sum'

      job_access_df = jt_emp_mx.groupby(['origin']).agg(agg_access_dict)
      return job_access_df

def export_job_access(job_access_df, population_assumptions, filename = 'MSMZone_Job_Access.csv'):
    value_cols = job_access_df.columns.tolist()
    job_access_pop_df = job_access_df.reset_index().merge(population_assumptions.rename(columns={'Zone': 'origin'}), on='origin')
    job_access_ijk = pd.melt(job_access_pop_df, id_vars = ['origin'], value_vars=value_cols)
    job_access_ijk['metrics'] = job_access_ijk['variable'].str.split("_").str[0]
    job_access_ijk['model'] = job_access_ijk['variable'].str.split("_").str[1]
    job_access_ijk['mode'] = job_access_ijk['variable'].str.split("_").str[2]
    job_access_ijk['trip_purpose'] = job_access_ijk['variable'].str.split("_").str[3]
    job_access_ijk['period_code'] = job_access_ijk['model'].str[2].astype(int)
    job_access_ijk['period'] = job_access_ijk['period_code'].map(REVERSE_PERIOD_MAP) 
    job_access_ijk['option'] = job_access_ijk['model'].str[-2:].astype(int)
    job_access_ijk = job_access_ijk.merge(population_assumptions, left_on = 'origin', right_on = 'Zone')
    job_access_ijk['pop*emp'] = job_access_ijk['Population'] * job_access_ijk['value']
    job_access_ijk.to_csv(os.path.join(THROUGHPUT_METRICS_DIR, filename), index=None)

In [15]:
## GC
def calculate_total_cost(trip_mx, gc_mx, trip_purpose_list = ['HBW', 'EB']):
    agg_dict = {}

    counter = 0
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            for mode in MODES_LIST:
                for trip_purpose in trip_purpose_list:
                    model_code = str(YEAR) + str(period_code) + str(scen_code)
                    trip_code = f'Trips_{model_code}_{mode}_{trip_purpose}'
                    gc_code = f'GC_{model_code}_{mode}_{trip_purpose}'
                    if (trip_code in trip_mx.columns) and (gc_code in gc_mx.columns):

                        filtered_trips = trip_mx[['origin', 'destination', trip_code]]
                        filtered_gc = gc_mx[['origin', 'destination', gc_code]]
                        temp_df = filtered_trips.merge(filtered_gc, on = ['origin', 'destination'])
                        temp_df[f'TotalCost_{model_code}_{mode}_{trip_purpose}'] = temp_df[gc_code] * temp_df[trip_code]
                        # temp_df = temp_df.drop(columns=[trip_code, time_code])

                        if counter == 0:
                            cost_df = temp_df.copy()
                        else:
                            cost_df = cost_df.merge(temp_df, on=['origin', 'destination'])

                        trip_code = f'Trips_{model_code}_{mode}_{trip_purpose}'
                        gc_code = f'TotalCost_{model_code}_{mode}_{trip_purpose}'
                        agg_dict[trip_code] = 'sum'
                        agg_dict[gc_code] = 'sum'

                        counter += 1

    cost_df = map_sector(cost_df)
    return cost_df

In [20]:
hbw_time_mx.columns = hbw_time_mx.columns.str.replace("_All", "_HBW")

In [21]:
# domin_all_time_mx.columns = domin_all_time_mx.columns.str.replace("_All", "_HBW")
# job_jt_access_df = get_job_accessibility(domin_all_time_mx, employment_assumptions, metric = 'Time', trip_purpose = 'HBW')
# export_job_access(job_jt_access_df, population_assumptions, filename = 'MSMZone_Job_Access_2018.csv')
job_jt_access_df = get_job_accessibility(hbw_time_mx, employment_assumptions, metric = 'Time', trip_purpose = 'HBW')
job_gc_access_df = get_job_accessibility(hbw_gc_mx, employment_assumptions, metric = 'GC', trip_purpose = 'HBW')
job_access_df = job_jt_access_df.join(job_gc_access_df)
export_job_access(job_access_df, population_assumptions, filename = 'MSMZone_Job_Access.csv')

In [70]:
job_jt_access_2018_df = job_jt_access_df[['AccToEmp-Time_18112_Car_HBW', 'AccToEmp-Time_18112_PT_HBW']].reset_index().merge(
    MSM_ZONES[['MSM2018', 'geometry']], left_on='origin', right_on='MSM2018'
).rename(columns={'AccToEmp-Time_18112_Car_HBW':'Car_JobAccess', 'AccToEmp-Time_18112_PT_HBW': 'PT_JobAccess'})

job_jt_access_2018_df['PT_Car_JobAccess_Ratio'] = job_jt_access_2018_df['PT_JobAccess'] / job_jt_access_2018_df['Car_JobAccess']
job_jt_access_2018_df = gpd.GeoDataFrame(job_jt_access_2018_df, geometry=job_jt_access_2018_df['geometry'], crs='epsg:2193')

OUTPUT_DIR = '/Users/tszchun.chow/Library/CloudStorage/OneDrive-Arup/Auckland ToU Charging/10 Strategic Case/Graphics/data/Future Year Networks'
job_jt_access_2018_df.to_file(f'{OUTPUT_DIR}/2018_JobAccess.shp')

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_2034/2527471123.py:9: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  job_jt_access_2018_df.to_file(f'{OUTPUT_DIR}/2018_JobAccess.shp')
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Car_JobAccess' to 'Car_JobAcc'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'PT_JobAccess' to 'PT_JobAcce'
  ogr_write(
/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'PT_Car_JobAccess_Ratio' to 'PT_Car_Job'
  ogr_write(


In [186]:
totalcost_df

,origin,destination,Trips_26116_Car_HBW,GC_26116_Car_HBW,TotalCost_26116_Car_HBW,Trips_26116_PT_HBW,GC_26116_PT_HBW,TotalCost_26116_PT_HBW,Trips_26176_Car_HBW,GC_26176_Car_HBW,...,Trips_26378_Car_HBW,GC_26378_Car_HBW,TotalCost_26378_Car_HBW,Trips_26378_PT_HBW,GC_26378_PT_HBW,TotalCost_26378_PT_HBW,origin_LBA,origin_sector,destination_LBA,destination_sector
0,1,1,215.065475,15.769327,3391.437844,0.000000e+00,4999.500000,0.000000e+00,215.000381,15.769352,...,254.563416,15.696969,3995.874050,0.000000,4999.500000,0.000000,Rodney,Rodney,Rodney,Rodney
1,2,1,19.055061,35.816154,682.479021,0.000000e+00,9999.000000,0.000000e+00,19.031487,35.816071,...,46.506344,35.907913,1669.945758,0.000000,9999.000000,0.000000,Rodney,Rodney,Rodney,Rodney
2,3,1,14.379362,37.991051,546.287075,0.000000e+00,9999.000000,0.000000e+00,14.362197,37.990936,...,7.563287,37.722939,285.309420,0.000000,9999.000000,0.000000,Rodney,Rodney,Rodney,Rodney
3,4,1,3.780962,40.083981,151.556026,0.000000e+00,9999.000000,0.000000e+00,3.776393,40.083881,...,27.973631,39.956520,1117.728945,0.000000,9999.000000,0.000000,Rodney,Rodney,Rodney,Rodney
4,5,1,24.023924,71.641716,1721.115131,0.000000e+00,9999.000000,0.000000e+00,23.994652,71.643257,...,28.230648,73.277527,2068.672070,0.000000,9999.000000,0.000000,Rodney,Rodney,Rodney,Rodney
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355211,592,596,0.000000,0.000000,0.000000,1.416476e-15,442.233093,6.264125e-13,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,9999.000000,0.000000,Franklin,Franklin,Aotea/Great Barrier,Other Areas
355212,593,596,0.000000,0.000000,0.000000,1.560663e-14,416.429504,6.499060e-12,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,9999.000000,0.000000,Franklin,Franklin,Aotea/Great Barrier,Other Areas
355213,594,596,0.000000,0.000000,0.000000,1.139578e-14,422.390930,4.813475e-12,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,9999.000000,0.000000,Franklin,Franklin,Aotea/Great Barrier,Other Areas
355214,595,596,0.000000,0.000000,0.000000,4.008296e-05,411.478638,1.649328e-02,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000343,572.842712,0.196587,Franklin,Franklin,Aotea/Great Barrier,Other Areas


In [22]:
totalcost_df = calculate_total_cost(hbw_trip_mx, hbw_gc_mx, trip_purpose_list=['HBW'])

trip_purpose = 'HBW'
agg_dict = {}
for period in PERIOD_MAP:
    period_code = PERIOD_MAP[period]
    for scenario in SCENARIO_MAP:
        scen_code = SCENARIO_MAP[scenario]
        model_code = year(scen_code) + str(period_code) + str(scen_code)
        modes_list = MODES_LIST
        for mode in modes_list:
            trip_code = f'Trips_{model_code}_{mode}_{trip_purpose}'
            if (trip_code in totalcost_df.columns):
                gc_code = f'TotalCost_{model_code}_{mode}_{trip_purpose}'
                agg_dict[trip_code] = 'sum'
                agg_dict[gc_code] = 'sum'

lba_cost_df = totalcost_df.groupby(['origin_LBA', 'destination_LBA']).agg(agg_dict)
value_cols = lba_cost_df.columns.tolist()
lba_cost_ijk = pd.melt(lba_cost_df.reset_index(), id_vars = ['origin_LBA', 'destination_LBA'], value_vars=value_cols)
lba_cost_ijk['metrics'] = lba_cost_ijk['variable'].str.split("_").str[0]
lba_cost_ijk['model'] = lba_cost_ijk['variable'].str.split("_").str[1]
lba_cost_ijk['mode'] = lba_cost_ijk['variable'].str.split("_").str[2]
lba_cost_ijk['trip_purpose'] = lba_cost_ijk['variable'].str.split("_").str[3]
lba_cost_ijk['period_code'] = lba_cost_ijk['model'].str[2].astype(int)
lba_cost_ijk['period'] = lba_cost_ijk['period_code'].map(REVERSE_PERIOD_MAP)
lba_cost_ijk['option'] = lba_cost_ijk['model'].str[-2:].astype(int)
lba_cost_ijk.to_csv(THROUGHPUT_METRICS_DIR + '/LBA_OD_GC.csv', index=None)

In [190]:
lba_cost_ijk['period'] = lba_cost_ijk['period_code'].map(REVERSE_PERIOD_MAP)
lba_cost_ijk['option'] = lba_cost_ijk['model'].str[-2:].astype(int)
lba_cost_ijk.to_csv(THROUGHPUT_METRICS_DIR + '/LBA_OD_GC.csv', index=None)

### Rough draft

In [7]:
def get_travel_time_mx(trip_mx, journeytime_mx, trip_purpose):
    agg_dict = {}

    counter = 0
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            for mode in MODES_LIST:
                model_code = year(scen_code) + str(period_code) + str(scen_code)
                trip_code = f'Trips_{model_code}_{mode}_{trip_purpose}'
                time_code = f'Time_{model_code}_{mode}_{trip_purpose}'

                if (trip_code in trip_mx.columns) and (time_code in journeytime_mx.columns):

                    filtered_trips = trip_mx[['origin', 'destination', trip_code]]
                    filtered_times = journeytime_mx[['origin', 'destination', time_code]]
                    temp_df = filtered_trips.merge(filtered_times, on = ['origin', 'destination'])
                    temp_df[f'PHT_{model_code}_{mode}_{trip_purpose}'] = temp_df[time_code] * temp_df[trip_code]
                    # temp_df = temp_df.drop(columns=[trip_code, time_code])

                    if counter == 0:
                        pht_df = temp_df.copy()
                    else:
                        pht_df = pht_df.merge(temp_df, on=['origin', 'destination'])

                    trip_code = f'Trips_{model_code}_{mode}_{trip_purpose}'
                    pht_code = f'PHT_{model_code}_{mode}_{trip_purpose}'
                    agg_dict[trip_code] = 'sum'
                    agg_dict[pht_code] = 'sum'

                    counter += 1

    pht_df = map_sector(pht_df)
    return pht_df

pht_df = get_travel_time_mx(hbw_veh_trips, hbw_veh_time, 'HBW')

In [53]:
## GC
def calculate_total_cost(trip_mx, gc_mx):
    agg_dict = {}

    counter = 0
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            if scen_code != 12:
                for mode in MODES_LIST:
                    for trip_purpose in ['HBW', "EB"]:
                        model_code = str(YEAR) + str(period_code) + str(scen_code)
                        trip_code = f'Trips_{model_code}_{mode}_{trip_purpose}'
                        gc_code = f'GC_{model_code}_{mode}_{trip_purpose}'

                        filtered_trips = trip_mx[['origin', 'destination', trip_code]]
                        filtered_gc = gc_mx[['origin', 'destination', gc_code]]
                        temp_df = filtered_trips.merge(filtered_gc, on = ['origin', 'destination'])
                        temp_df[f'TotalCost_{model_code}_{mode}_{trip_purpose}'] = temp_df[gc_code] * temp_df[trip_code]
                        # temp_df = temp_df.drop(columns=[trip_code, time_code])

                        if counter == 0:
                            cost_df = temp_df.copy()
                        else:
                            cost_df = cost_df.merge(temp_df, on=['origin', 'destination'])

                        trip_code = f'Trips_{model_code}_{mode}_{trip_purpose}'
                        gc_code = f'TotalCost_{model_code}_{mode}_{trip_purpose}'
                        agg_dict[trip_code] = 'sum'
                        agg_dict[gc_code] = 'sum'

                        counter += 1

    cost_df = map_sector(cost_df)
    return cost_df, agg_dict


In [52]:
cost_df

,origin,destination,Trips_26116_Car_HBW,GC_26116_Car_HBW,TotalCost_26116_Car_HBW,Trips_26116_Car_EB,GC_26116_Car_EB,TotalCost_26116_Car_EB,Trips_26116_PT_HBW,GC_26116_PT_HBW,...,Trips_26374_PT_HBW,GC_26374_PT_HBW,TotalCost_26374_PT_HBW,Trips_26374_PT_EB,GC_26374_PT_EB,TotalCost_26374_PT_EB,origin_LBA,origin_sector,destination_LBA,destination_sector
0,1,1,215.065475,15.769327,3391.437844,34.608597,13.512896,467.662355,0.000000e+00,4999.500000,...,0.000000,4999.500000,0.000000,0.000000e+00,4999.500000,0.000000e+00,Rodney,Other Areas,Rodney,Other Areas
1,2,1,19.055061,35.816154,682.479021,1.488872,30.284849,45.090276,0.000000e+00,9999.000000,...,0.000000,9999.000000,0.000000,0.000000e+00,9999.000000,0.000000e+00,Rodney,Other Areas,Rodney,Other Areas
2,3,1,14.379362,37.991051,546.287075,0.563615,32.018589,18.046165,0.000000e+00,9999.000000,...,0.000000,9999.000000,0.000000,0.000000e+00,9999.000000,0.000000e+00,Rodney,Other Areas,Rodney,Other Areas
3,4,1,3.780962,40.083981,151.556026,0.634810,33.549103,21.297315,0.000000e+00,9999.000000,...,0.000000,9999.000000,0.000000,0.000000e+00,9999.000000,0.000000e+00,Rodney,Other Areas,Rodney,Other Areas
4,5,1,24.023924,71.641716,1721.115131,2.892753,59.748356,172.837229,0.000000e+00,9999.000000,...,0.000000,9999.000000,0.000000,0.000000e+00,9999.000000,0.000000e+00,Rodney,Other Areas,Rodney,Other Areas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355211,592,596,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.416476e-15,442.233093,...,0.000000,9999.000000,0.000000,1.567391e-18,9999.000000,1.567235e-14,Franklin,Other Areas,Aotea/Great Barrier,Other Areas
355212,593,596,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.560663e-14,416.429504,...,0.000000,9999.000000,0.000000,2.982887e-18,9999.000000,2.982589e-14,Franklin,Other Areas,Aotea/Great Barrier,Other Areas
355213,594,596,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.139578e-14,422.390930,...,0.000000,9999.000000,0.000000,3.076190e-18,9999.000000,3.075882e-14,Franklin,Other Areas,Aotea/Great Barrier,Other Areas
355214,595,596,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.008296e-05,411.478638,...,0.000353,572.842163,0.202352,3.572817e-03,470.012482,1.679269e+00,Franklin,Other Areas,Aotea/Great Barrier,Other Areas


In [26]:
lba_cost_df = cost_df.groupby(['origin_LBA', 'destination_LBA']).agg(agg_dict)
value_cols = lba_cost_df.columns.tolist()
lba_cost_ijk = pd.melt(lba_cost_df.reset_index(), id_vars = ['origin_LBA', 'destination_LBA'], value_vars=value_cols)
lba_cost_ijk['metrics'] = lba_cost_ijk['variable'].str.split("_").str[0]
lba_cost_ijk['model'] = lba_cost_ijk['variable'].str.split("_").str[1]
lba_cost_ijk['mode'] = lba_cost_ijk['variable'].str.split("_").str[2]
lba_cost_ijk['trip_purpose'] = lba_cost_ijk['variable'].str.split("_").str[3]
lba_cost_ijk['period'] = lba_cost_ijk['model'].str[2].astype(int)
lba_cost_ijk['option'] = lba_cost_ijk['model'].str[-2:].astype(int)
lba_cost_ijk.to_csv(THROUGHPUT_METRICS_DIR + '/LBA_OD_GC.csv', index=None)

In [30]:
sector_od_df = pht_df.groupby(['origin_sector', 'destination_sector']).agg(agg_dict).reset_index()
lba_od_df = pht_df.groupby(['origin_LBA', 'destination_LBA']).agg(agg_dict).reset_index()

for period in PERIOD_MAP:
    period_code = PERIOD_MAP[period]
    for scenario in SCENARIO_MAP:
        scen_code = SCENARIO_MAP[scenario]
        for mode in MODES_LIST:
            model_code = str(YEAR) + str(period_code) + str(scen_code)
            jt_code = f'Time_{model_code}_{mode}'
            trip_code = f'Trips_{model_code}_{mode}'
            pht_code = f'PHT_{model_code}_{mode}'
            sector_od_df[jt_code] = sector_od_df[pht_code] / sector_od_df[trip_code]
            lba_od_df[jt_code] = lba_od_df[pht_code] / lba_od_df[trip_code]

lba_od_df 

,origin_LBA,destination_LBA,Trips_26116_Car,PHT_26116_Car,Trips_26116_PT,PHT_26116_PT,Trips_26176_Car,PHT_26176_Car,Trips_26176_PT,PHT_26176_PT,...,Time_26364_Car,Time_26364_PT,Time_26366_Car,Time_26366_PT,Time_26370_Car,Time_26370_PT,Time_26372_Car,Time_26372_PT,Time_26374_Car,Time_26374_PT
0,Albert - Eden,Albert - Eden,12393.278018,90737.639027,946.715719,27488.401146,12369.937475,90013.647042,947.556516,27475.200975,...,6.806637,28.245826,6.961084,28.386559,7.072212,27.722208,6.732912,28.171654,6.831305,28.233617
1,Albert - Eden,Aotea/Great Barrier,0.000000,0.000000,3.592781,398.404285,0.000000,0.000000,3.606671,399.812182,...,NaN,109.308892,NaN,109.357331,NaN,109.034330,NaN,109.228658,NaN,109.291074
2,Albert - Eden,Devonport - Takapuna,318.737273,7435.188946,156.505452,8419.871433,313.563185,7298.827941,156.848498,8411.125924,...,26.980924,53.821389,27.351004,54.632797,30.306999,53.775813,28.964951,54.152796,27.021250,54.319763
3,Albert - Eden,Franklin,163.918329,8078.982466,11.702172,1264.389727,163.699741,8070.180041,11.719285,1265.353653,...,60.013868,101.005012,72.578411,101.190996,73.126921,100.942204,72.363424,100.937001,73.366714,101.116962
4,Albert - Eden,Henderson - Massey,1069.354701,20084.487082,169.401375,9350.309501,1067.311706,20018.745608,169.194508,9329.144380,...,28.475399,57.917385,34.693733,59.150730,32.978852,57.696344,26.264077,57.844011,34.296909,58.519028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,Whau,Rodney,256.131566,9257.748924,10.747274,1195.851256,257.209247,9310.514813,10.779822,1198.857972,...,37.432418,101.196129,45.636527,102.060541,45.102835,102.232108,38.308008,102.263499,45.581378,102.099960
396,Whau,Upper Harbour,481.370041,14408.699887,102.108140,8320.148010,483.464614,14470.737556,101.900215,8288.465142,...,24.736643,78.199003,33.465591,79.882596,32.328762,79.580268,25.538782,79.662890,33.457269,79.874599
397,Whau,Waitakere Ranges,1760.562663,18988.740117,101.409699,4873.735977,1761.524029,19024.239929,101.711560,4888.484751,...,14.764089,49.159577,14.913683,49.166767,14.743804,48.983775,14.428627,48.977960,14.918084,49.128609
398,Whau,Waitemata,3841.394081,132110.976256,4034.572152,207905.890191,3831.749158,130870.720878,4007.179576,205831.096481,...,20.401828,50.881255,21.880368,51.024511,17.963653,50.783122,17.630546,50.804854,21.660290,51.004375


In [39]:
sector_od_df.to_parquet(THROUGHPUT_METRICS_DIR + '/sector_OD.parquet')

In [ ]:
pht_df.to_parquet(THROUGHPUT_METRICS_DIR + '/msmzone_OD.parquet')

In [9]:
def export_geoparquet(geo_package, output_path, filename, id = 'ID'):
    geo_package.to_crs("EPSG:4326", inplace=True)
    geo_package['wkt'] = geo_package['geometry'].to_wkt()
    
    geo_dataframe = gpd.GeoDataFrame()
    geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])    
    geo_dataframe = gpd.GeoDataFrame(geo_dataframe, geometry="geometry")
    geo_dataframe[id] = geo_package[id]

    geo_parquet = geo_dataframe.merge(geo_package[[i for i in geo_package.columns if i not in ["geometry", "wkt"]]],
                                    left_on=id, right_on = id)

    geo_parquet = geo_parquet.dropna(subset=['geometry'])
    if output_path is not None and filename is not None:
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        geo_parquet.fillna(0, inplace=True)
        geo_parquet.to_parquet(f'{output_path}/{filename}', compression = 'zstd')

pht_origin_df = pht_df.groupby(['origin', 'origin_sector']).agg(agg_dict).reset_index()
remove_cols = []
for period in PERIOD_MAP:
    period_code = PERIOD_MAP[period]
    for scenario in SCENARIO_MAP:
        scen_code = SCENARIO_MAP[scenario]
        for mode in MODES_LIST:
            model_code = str(YEAR) + str(period_code) + str(scen_code)
            trip_code = f'Trips_{model_code}_{mode}'
            pht_code = f'PHT_{model_code}_{mode}'
            wtav_jt_code = f'AvgTime_{model_code}_{mode}'
            
            pht_origin_df[wtav_jt_code] = pht_origin_df[pht_code] / pht_origin_df[trip_code]
            remove_cols.append(pht_code)
pht_origin_df = pht_origin_df.drop(columns=remove_cols)
pht_origin_df = gpd.GeoDataFrame(pht_origin_df.merge(MSM_ZONES[['MSM2018', 'geometry', 'LBA_Sector']], 
                                                     left_on='origin', 
                                                     right_on = 'MSM2018', 
                                                     how='left'),
                                                     geometry='geometry',
                                                     crs='epsg:2193').to_crs('epsg:2193')

export_geoparquet(pht_origin_df, output_path=THROUGHPUT_METRICS_DIR, filename='JourneyTime_Origin.parquet', id = 'MSM2018')

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_24155/3767556367.py:6: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])


In [10]:
pht_origin_df

,origin,origin_sector,Trips_26116_Car,Trips_26116_PT,Trips_26176_Car,Trips_26176_PT,Trips_26150_Car,Trips_26150_PT,Trips_26152_Car,Trips_26152_PT,...,AvgTime_26370_Car,AvgTime_26370_PT,AvgTime_26372_Car,AvgTime_26372_PT,AvgTime_26374_Car,AvgTime_26374_PT,MSM2018,geometry,LBA_Sector,wkt
0,1,Other Areas,998.408801,0.000000,998.430189,0.000000,998.770444,0.000000,1000.659853,0.000000,...,23.034215,NaN,22.904792,NaN,22.964004,NaN,1,"MULTIPOLYGON (((174.76852 -36.31946, 174.76859...",Other Areas,"MULTIPOLYGON (((174.768516 -36.319457, 174.768..."
1,2,Other Areas,391.026338,13.686235,391.038093,13.665237,391.034102,13.901942,391.574760,13.978206,...,14.829508,61.040778,14.747197,60.922451,14.776701,60.783736,2,"POLYGON ((174.51452 -36.27879, 174.51479 -36.2...",Other Areas,"POLYGON ((174.51452 -36.278787, 174.514795 -36..."
2,3,Other Areas,245.534137,10.070281,245.537243,10.060241,245.552933,10.153015,245.931599,10.156683,...,16.189002,82.306167,16.042715,81.790993,16.079432,81.333653,3,"POLYGON ((174.52611 -36.25191, 174.52617 -36.2...",Other Areas,"POLYGON ((174.526113 -36.25191, 174.526169 -36..."
3,4,Other Areas,122.237983,3.922730,122.243189,3.918662,122.340376,3.955399,122.611539,3.956422,...,18.120312,70.691732,18.053812,70.609384,18.082051,70.550511,4,"POLYGON ((174.53181 -36.29209, 174.53206 -36.2...",Other Areas,"POLYGON ((174.531813 -36.292092, 174.532063 -3..."
4,5,Other Areas,1318.320285,0.000000,1318.253632,0.000000,1318.657666,0.000000,1321.257097,0.000000,...,28.185603,NaN,27.991931,NaN,28.064299,NaN,5,"MULTIPOLYGON (((174.39477 -36.42154, 174.39496...",Other Areas,"MULTIPOLYGON (((174.394768 -36.421543, 174.394..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,592,Other Areas,935.317050,90.519578,935.874109,89.007867,935.623884,91.085089,935.506031,91.128709,...,21.575542,0.000000,21.323600,0.000000,21.427654,0.000000,592,"POLYGON ((174.62844 -37.0361, 174.62852 -37.03...",Other Areas,"POLYGON ((174.628443 -37.036104, 174.628516 -3..."
592,593,Other Areas,1691.424352,284.090211,1694.089025,278.708814,1694.023219,283.759816,1693.727191,283.698821,...,30.368436,0.000000,29.921808,0.000000,30.133398,0.000000,593,"POLYGON ((175.0736 -37.04027, 175.07376 -37.04...",Other Areas,"POLYGON ((175.073605 -37.040267, 175.07376 -37..."
593,594,Other Areas,1965.329877,218.468385,1965.166995,218.795433,1967.758672,222.183560,1967.312947,222.052089,...,22.860864,0.000000,22.460286,0.000000,22.631486,0.000000,594,"POLYGON ((174.97502 -37.20107, 174.97964 -37.2...",Other Areas,"POLYGON ((174.975021 -37.20107, 174.979635 -37..."
594,595,Other Areas,1991.719207,319.550577,1991.156139,320.440368,1991.028667,324.359751,1990.896306,324.600590,...,16.049249,148.200465,15.770040,148.087227,15.889659,148.421872,595,"POLYGON ((174.9476 -37.21438, 174.94854 -37.21...",Other Areas,"POLYGON ((174.947602 -37.214377, 174.948545 -3..."


In [17]:
pht_destination_df = pht_df.groupby(['destination', 'destination_sector']).agg(agg_dict).reset_index()
remove_cols = []
for period in PERIOD_MAP:
    period_code = PERIOD_MAP[period]
    for scenario in SCENARIO_MAP:
        scen_code = SCENARIO_MAP[scenario]
        for mode in MODES_LIST:
            model_code = str(YEAR) + str(period_code) + str(scen_code)
            trip_code = f'Trips_{model_code}_{mode}'
            pht_code = f'PHT_{model_code}_{mode}'
            wtav_jt_code = f'AvgTime_{model_code}_{mode}'
            
            pht_destination_df[wtav_jt_code] = pht_destination_df[pht_code] / pht_destination_df[trip_code]
            remove_cols.append(pht_code)
pht_destination_df = pht_destination_df.drop(columns=remove_cols)
pht_destination_df = gpd.GeoDataFrame(pht_destination_df.merge(MSM_ZONES[['MSM2018', 'geometry', 'LBA_Sector']], 
                                                     left_on='destination', 
                                                     right_on = 'MSM2018', 
                                                     how='left'),
                                                     geometry='geometry',
                                                     crs='epsg:2193')

export_geoparquet(pht_destination_df, output_path=THROUGHPUT_METRICS_DIR, filename='JourneyTime_Destination.parquet', id = 'MSM2018')

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_13045/3767556367.py:6: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])


In [18]:

pht_sector_df = pht_df.groupby(['origin_sector', 'destination_sector']).agg(agg_dict).reset_index()
for period in PERIOD_MAP:
    period_code = PERIOD_MAP[period]
    for scenario in SCENARIO_MAP:
        scen_code = SCENARIO_MAP[scenario]
        for mode in MODES_LIST:
            model_code = str(YEAR) + str(period_code) + str(scen_code)
            trip_code = f'Trips_{model_code}_{mode}'
            pht_code = f'PHT_{model_code}_{mode}'
            wtav_jt_code = f'AvgTime_{model_code}_{mode}'
            
            pht_sector_df[wtav_jt_code] = pht_sector_df[pht_code] / pht_sector_df[trip_code]
            remove_cols.append(pht_code)
pht_sector_df = pht_sector_df.drop(columns=remove_cols)
pht_sector_df.to_parquet(THROUGHPUT_METRICS_DIR + '/sector_OD_journeytime.parquet')

In [51]:
MSM_ZONES

,MSM2018,Shape_Leng,Shape_Area,geometry,Sector,Sector_Name,Local Board Area,LBA_2,LBA_Sector
0,1,146256.701398,3.795254e+08,"MULTIPOLYGON Z (((1758754.382 5979166.71 0, 17...",1,Rodney,Rodney,Rodney,Other Areas
1,2,9244.130579,2.656092e+06,"POLYGON Z ((1736022.794 5984065.325 0, 1736047...",1,Rodney,Rodney,Rodney,Other Areas
2,3,15582.018789,1.079087e+07,"POLYGON Z ((1737111.112 5987030.491 0, 1737116...",1,Rodney,Rodney,Rodney,Other Areas
3,4,16746.451008,7.902337e+06,"POLYGON Z ((1737552.719 5982564.937 0, 1737574...",1,Rodney,Rodney,Rodney,Other Areas
4,5,300894.761365,5.823116e+08,"MULTIPOLYGON Z (((1725038.396 5968390.919 0, 1...",1,Rodney,Rodney,Rodney,Other Areas
...,...,...,...,...,...,...,...,...,...
591,592,141580.175621,2.213256e+08,"POLYGON Z ((1744829.308 5899882.633 0, 1744835...",17,South Auckland,Franklin,Franklin,Other Areas
592,593,78074.621802,1.926560e+08,"POLYGON Z ((1784415.005 5898649.945 0, 1784428...",17,South Auckland,Franklin,Franklin,Other Areas
593,594,35884.401338,3.704136e+07,"POLYGON Z ((1775275.229 5880995.042 0, 1775665...",18,Other Areas,Franklin,Franklin,Other Areas
594,595,40658.032227,3.547091e+07,"POLYGON Z ((1772811.262 5879568.965 0, 1772894...",18,Other Areas,Franklin,Franklin,Other Areas


In [ ]:
counter = 0

for period in PERIOD_MAP:
    period_code = PERIOD_MAP[period]
    for scenario in SCENARIO_MAP:
        scen_code = SCENARIO_MAP[scenario]
        model_code = str(YEAR) + str(period_code) + str(scen_code)
        metric_code = f'Time_{model_code}_Car'

        if scenario not in [x for x in BASE_SCENARIO_MAP]:
            if scenario == 'Option 3C':
                base_model_code = str(YEAR) + str(period_code) + str(SCENARIO_MAP['Do Minimum 3C'])
            else:
                base_model_code = str(YEAR) + str(period_code) + str(SCENARIO_MAP['Do Minimum'])

            temp_df = 

In [178]:
pht_df

,origin,destination,Trips_26116_Car,Time_26116_Car,PHT_26116_Car,Trips_26116_PT,Time_26116_PT,PHT_26116_PT,Trips_26176_Car,Time_26176_Car,...,Time_26372_PT,PHT_26372_PT,Trips_26374_Car,Time_26374_Car,PHT_26374_Car,Trips_26374_PT,Time_26374_PT,PHT_26374_PT,origin_sector,destination_sector
0,1,1,510.729523,11.688956,5969.895052,0.000000e+00,0.000000,0.000000e+00,510.625031,11.688981,...,0.000000,0.000000,766.369324,11.614069,8900.666194,0.000000e+00,0.000000,0.000000,Rodney,Rodney
1,2,1,28.552387,25.813732,737.043676,0.000000e+00,0.000000,0.000000e+00,28.519699,25.813648,...,0.000000,0.000000,78.996910,25.904011,2046.336810,0.000000e+00,0.000000,0.000000,Rodney,Rodney
2,3,1,16.809258,27.190876,457.058437,0.000000e+00,0.000000,0.000000e+00,16.789474,27.190758,...,0.000000,0.000000,11.933378,26.923866,321.292679,0.000000e+00,0.000000,0.000000,Rodney,Rodney
3,4,1,7.669829,28.266777,216.801343,0.000000e+00,0.000000,0.000000e+00,7.661741,28.266678,...,0.000000,0.000000,38.930183,28.140530,1095.515980,0.000000e+00,0.000000,0.000000,Rodney,Rodney
4,5,1,35.542103,50.134605,1781.889300,0.000000e+00,0.000000,0.000000e+00,35.502350,50.136150,...,0.000000,0.000000,48.889103,51.765026,2530.745689,0.000000e+00,0.000000,0.000000,Rodney,Rodney
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355211,592,596,0.000000,0.000000,0.000000,1.798056e-15,193.370483,3.476910e-13,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.567391e-18,0.000000,0.000000,South Auckland,Other Areas
355212,593,596,0.000000,0.000000,0.000000,1.950911e-14,180.468689,3.520784e-12,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.982887e-18,0.000000,0.000000,South Auckland,Other Areas
355213,594,596,0.000000,0.000000,0.000000,1.430679e-14,191.151077,2.734759e-12,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.076190e-18,0.000000,0.000000,Other Areas,Other Areas
355214,595,596,0.000000,0.000000,0.000000,7.683419e-03,185.694916,1.426772e+00,0.000000,0.000000,...,264.369293,3.505694,0.000000,0.000000,0.000000,1.308266e-02,264.368622,3.458644,Other Areas,Other Areas


In [149]:
output_dir = '../outputs/throughput_metrics'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
trip_mx.to_parquet(f'{output_dir}/trips_mx.parquet')
journeytime_mx.to_parquet(f'{output_dir}/journey_time_mx.parquet')
tolls_mx.to_parquet(f'{output_dir}/tolls_mx.parquet')

KeyboardInterrupt: 

In [192]:
pht_sector_df.reset_index().to_csv(f'{output_dir}/pht_bysector.csv', index=False)